This assumes you

- Have installed PyRosetta4
- Are using Anaconda's Python Package
- Have installed RISE (see link below)

conda install -c damianavila82 rise

https://github.com/damianavila/RISE
    
Restart the notebook, you should now see a histogram. Click on it (above this slide in your toolbar) to enter the slide show

<img src='Screenshot from 2017-02-09 11-05-05.png'/>

# Enzyme Design in PyRosetta

<br></br>

## Steve Bertolani
## Winter RosettaCon


# Classic

C++


./enzyme_design.default.linuxgccrelease –database ./../../database -s mypdb.pdb -extra_res_fa my.params -ex1 -ex2 -enzdesflasgs -nstruct 10 -ncycles 20 -trials

Recompile to change,  get designed PDBs as output
Black box



# New(er)

XML


./rosetta_scripts.default.linuxgccrelease –database ./../../database @enzymedesign.flags –parser:protocol

Can easily add movers, must use –jd2:enzdes to get cst energy terms broken down, PDB’s output
Black box (but you can add other boxes before and after) or read the rosetta trace


# Python


Python enzymedesign.py enzdesflags


No need to output PDBs, easily add your own movers, extract energies, process further down the pipeline, autocomplete commands
Semi transparent black box, with inspection of most things available (TO, PackerTask, Selections)


## This uses PyRosetta 4.0 bindings

## Enyzme Design with Pyrosetta

Thanks to the new version of the PyRosetta bindings (nice work Sergey!) most of the enzdes code now works in python. There are a few things to note along the way. This demo will show you how to run enzyme design in PyRosetta using the files located at https://github.com/dacarlin/bagel-foldit. That location has a PDB file, an enzyme design file, params file (and rotamers)

Note: Try "git clone https://github.com/dacarlin/bagel-foldit" in the same directory that you have this notebook. Also, you must have already downloaded and pip installed pyrosetta4 bindings to your python. This assumes you have Anaconda's python 2.7 installed as well.

You can always pass your extra desired options into the pyrosetta.init string

This is not entirely reproducing the xml as of yet.

# Start Rosetta

In [ ]:
import rosetta
import pyrosetta
pyrosetta.init('-extra_res_fa ./bagel-foldit/cid92930.params -run:preserve_header T')

In [ ]:
#print pyrosetta.rosetta.basic.options.get_boolean_option("run:preserve_header")
print pyrosetta.rosetta.basic.options.get_boolean_option('packing:ex1')
pyrosetta.rosetta.basic.options.set_boolean_option('packing:ex1',True)
print pyrosetta.rosetta.basic.options.get_boolean_option('packing:ex1')

# Setup Globals <- for now

In [ ]:
# This must be turned on globally for the enable_cst_scorefunction to turn on the fnr term (which is actually res_type_constraint, see protocols:protein_design_interface:design_utils)
# and also see protocols.enzdes.endes_utils
# Note, how we can print the value before and after it's set to verify our changes

print pyrosetta.rosetta.basic.options.get_real_option('enzdes:favor_native_res')
pyrosetta.rosetta.basic.options.set_real_option('enzdes:favor_native_res',2.0)

print pyrosetta.rosetta.basic.options.get_real_option('enzdes:lig_packer_weight')
pyrosetta.rosetta.basic.options.set_real_option('enzdes:lig_packer_weight',1.5)
#print pyrosetta.rosetta.basic.options.get_real_option('enzdes:lig_packer_weight')

## These must be set globally for the detect_ligand_interace to work, there should be getters and setters, but 
## as of today, there aren't any
print pyrosetta.rosetta.basic.options.get_real_option('enzdes:cut1')
print pyrosetta.rosetta.basic.options.get_real_option('enzdes:cut2')
print pyrosetta.rosetta.basic.options.get_real_option('enzdes:cut3')
print pyrosetta.rosetta.basic.options.get_real_option('enzdes:cut4')
print pyrosetta.rosetta.basic.options.get_boolean_option('enzdes:detect_design_interface')

pyrosetta.rosetta.basic.options.set_real_option('enzdes:cut1',10.0)
pyrosetta.rosetta.basic.options.set_real_option('enzdes:cut2',10.0)
pyrosetta.rosetta.basic.options.set_real_option('enzdes:cut3',12.0)
pyrosetta.rosetta.basic.options.set_real_option('enzdes:cut4',14.0)
pyrosetta.rosetta.basic.options.set_boolean_option('enzdes:detect_design_interface',True)

In [ ]:
print pyrosetta.rosetta.basic.options.get_real_option('enzdes:cut1')
print pyrosetta.rosetta.basic.options.get_real_option('enzdes:cut2')
print pyrosetta.rosetta.basic.options.get_real_option('enzdes:cut3')
print pyrosetta.rosetta.basic.options.get_real_option('enzdes:cut4')

# Make a pose

In [ ]:
p = pyrosetta.pose_from_file('./bagel-foldit/bagel.pdb')

In [ ]:
# Optional, if you want to watch in Pymol
pm = rosetta.protocols.moves.PyMolMover()
pm.keep_history(True)
pm.apply(p) ## Note, this started the Pymol counter to state 1 (view at bottom right hand corner)

# Visualization

The next line gets your browser to download the bio-pv javascript code so that we can construct a pv object

In [ ]:
%%javascript
require.config({
  paths: {
      pv: 'http://rawgit.com/biasmv/pv/master/bio-pv.min'
  }
});

The next cell writes html to construct a pvviewer object and then attaches that to the output from the notebook cell. We need the unique number so that if we call this function in another cell, it will create a new pv viewer, instead of attaching to the old one

In [ ]:
from IPython.display import display, Javascript

def viewmyprotein( pdbfilename, number ):
    display(Javascript("""

element.append("<div id='viewerNUMBER' style='width: 500dpx; height: 500dpx'></div>");

require(['pv'], function (pv){

    var parent = document.getElementById('viewerNUMBER');

    var viewer = pv.Viewer(parent, { width : 900, height : 450, antialias : true });
    pv.io.fetchPdb('MYPDB', function(structure) {

        viewer.on('viewerReady', function() {
            var ligand = structure.select({rnames : ['LG1']});
            viewer.ballsAndSticks('ligand', ligand);
            viewer.cartoon('protein', structure);
            viewer.autoZoom();
            });
    });
});

""".replace("MYPDB",pdbfilename).replace('NUMBER',number)))

In [ ]:
viewmyprotein('2jie.pdb',"1")

In [ ]:
viewmyprotein('./bagel-foldit/bagel.pdb',"2")

# Grab a Scorefunction

In [ ]:
## Setup Scorefunctions

sfxn = pyrosetta.get_fa_scorefxn()
rosetta.protocols.enzdes.enzutil.enable_constraint_scoreterms(sfxn)

soft_rep = rosetta.core.scoring.ScoreFunctionFactory.create_score_function("soft_rep")

     <SCOREFXNS>
               <myscore weights=talaris2013_cst.wts/>
     </SCOREFXNS>

In [ ]:
mm = pyrosetta.MoveMap()
mm.show(p.total_residue())

# Add Csts, setup movers

In [ ]:

addcsts = rosetta.protocols.enzdes.AddOrRemoveMatchCsts()
addcsts.cstfile('bagel-foldit/cid92930.enzdes.cst')
addcsts.set_cst_action( rosetta.protocols.enzdes.CstAction.ADD_NEW )
addcsts.apply(p)


predock = rosetta.protocols.enzdes.PredesignPerturbMover()
predock.trans_magnitude(0.1)
predock.rot_magnitude(2)
predock.set_ligand(446)

     <MOVERS>

             <AddOrRemoveMatchCsts name=cstadd cst_instruction=add_new/>
             
             <PredesignPerturbMover name=predock trans_magnitude=0.1 rot_magnitude=1 dock_trials=500/>

# Setup EnzDes movers

In [ ]:
enzdes = rosetta.protocols.enzdes.EnzRepackMinimize()
enzdes.set_scorefxn_minimize(sfxn)
enzdes.set_scorefxn_repack(soft_rep)
enzdes.set_min_lig(True)
enzdes.set_min_rb(True)
enzdes.set_min_sc(True)
enzdes.set_design(True)

enzdes_wbb = rosetta.protocols.enzdes.EnzRepackMinimize()
enzdes_wbb.set_scorefxn_minimize(sfxn)
enzdes_wbb.set_min_lig(True)
enzdes_wbb.set_min_rb(True)
enzdes_wbb.set_min_sc(True)
enzdes_wbb.set_design(True)
enzdes_wbb.set_scorefxn_repack(soft_rep)
enzdes_wbb.set_min_bb(True)

sfxn(p)

              <EnzRepackMinimize name=desmin_nobb design=1 repack_only=0 scorefxn_minimize=myscore scorefxn_repack=soft_rep minimize_rb=1 minimize_sc=1 minimize_bb=0 cycles=1 minimize_lig=1 min_in_stages=0 backrub=0 task_operations=edto,limchi2,catres/>
             
             <EnzRepackMinimize name=desmin_wbb design=1 repack_only=0 scorefxn_minimize=myscore scorefxn_repack=soft_rep minimize_rb=1 minimize_sc=1 minimize_bb=1 cycles=1 minimize_lig=1 min_in_stages=0 backrub=0 task_operations=edto,limchi2,catres/>

# Score it ( populates nbr graph)

In [ ]:
sfxn(p)

# Setup Task Operations

In [ ]:
tf = rosetta.core.pack.task.TaskFactory()

# Sets up the packing/design shells from the global options set above
dp = rosetta.protocols.enzdes.DetectProteinLigandInterface()
dp.init_from_options()

limchi2 = rosetta.protocols.toolbox.task_operations.LimitAromaChi2Operation()

# This restricts the residues define in the constraint file to only be allowed to pack, not designable
canttouchcatres = rosetta.protocols.enzdes.SetCatalyticResPackBehavior()
canttouchcatres.set_fix_catalytic_aa(False) ## seems to freeze them, no repacking if set to True

# Push back all of our TO's onto the task factory (see Design Patterns on Factorys)
tf.push_back(dp)
tf.push_back(canttouchcatres)
tf.push_back(limchi2)

# Create a packer task, specifically for the cstopt mover to work
pt = tf.create_task_and_apply_taskoperations(p)

enzdes.task_factory( tf ) # sets the enzdes movers to use our task factory set with our task operations 
enzdes_wbb.task_factory( tf )

    <TASKOPERATIONS>
               <DetectProteinLigandInterface name=edto design=1 cut1=6.0 cut2=8.0 cut3=10.0 cut4=12.0/>
               <DetectProteinLigandInterface name=edto_repack design=1 cut1=6.0 cut2=6.0 cut3=10.0 cut4=12.0/>
               <LimitAromaChi2 name=limchi2/>
               <SetCatalyticResPackBehavior name=catres fix_catalytic_aa=0/>
               <SetCatalyticResPackBehavior name=fixcat fix_catalytic_aa=1/>
                <ProteinLigandInterfaceUpweighter name=up interface_weight=1.5/>
     </TASKOPERATIONS>
     

In [ ]:
# Just leaving this here as an example, but this allows for manual inspection of your packer task 
# after all of the task operations have been applied. You could also apply one at a time to verify how
# they are working and that they are indeed working.

for i in xrange(1,p.total_residue()+1):
    print i,pt.being_designed(i),pt.being_packed(i)

In [ ]:
print pt

# Hack Cst Opt to work (to be fixed)

In [ ]:
# Mover sub-classing -----------------------------------                                                         
# Thank you Sergey!
class My_New_Mover(rosetta.protocols.moves.Mover):
    def __init__(self,sfxn,pt):
        print( 'My_New_Mover.__init__...' )
        rosetta.protocols.moves.Mover.__init__(self)
        self.sfxn = sfxn
        self.pt = pt
        
    def get_name(self): return 'My_New_Mover'

    def apply(self, p):

        cstopt = rosetta.protocols.enzdes.EnzdesBaseProtocol()
        cstopt.set_scorefxn( self.sfxn )
        cstopt.set_minimize_options(True, False, True, True) # check this fn signature for details
        
        # this actuall runs the minimizer !
        cstopt.cst_minimize(p, self.pt, True)

cstopt = My_New_Mover(sfxn,pt)

## Note that a mover must have an apply function that takes a pose and does something to it
## in this case, cst opt derives from Enzdesbase protocol, but in the xml it's call from an option in
## the enzdesrepackmin mover... blah blah blah, long story short, this ends up working in pyrosetta since
## there are currently no setters for this option in the enzrepackmin mover... this needs to be fixed

             <EnzRepackMinimize name=cstopt cst_opt=1 minimize_rb=1 minimize_sc=1 minimize_bb=0 cycles=1 min_in_stages=0 minimize_lig=1/>

# Add movers in sequence

In [ ]:
## Now we string all of our movers together so that one mover can be fed into the generic monte carlo
#test_pose.dump_pdb('out.pdb')
#viewmyprotein('out.pdb')
    
parsed = rosetta.protocols.moves.SequenceMover()

parsed.add_mover(predock)

parsed.add_mover(cstopt)

parsed.add_mover(enzdes)

parsed.add_mover(enzdes_wbb)

             <ParsedProtocol name=dock_des>
             	<Add mover=predock/>
                <Add mover=cstopt/>
                <Add mover=desmin_wbb/>
             </ParsedProtocol>

# Add sequence to MC mover

In [ ]:
mc = rosetta.protocols.simple_moves.GenericMonteCarloMover()
mc.set_drift(True) # this sets drift for the maxtrials (not technically mc anymore)
mc.set_maxtrials(1)  # CHANGE THIS TO 10 for real runs
mc.set_sampletype('low')
mc.set_temperature(0.6)
mc.set_mover(parsed)
mc.set_scorefxn(sfxn)

	     <GenericMonteCarlo name=multi_dd mover_name=dock_des filter_name=allcst trials=10 sample_type=low temperature=0.6 drift=1/>

</MOVERS>

In [ ]:
# This is how you would normally run for nstruct 10
# Note: This is not actually necessary depending on your goals, you can
# also mc.apply(p), or store the energies over the simulation, or p.dump_pdb() etc.
nstruct = 2
job_output = 'test_output'
jd = pyrosetta.PyJobDistributor(job_output, nstruct, sfxn)
temp_pose = rosetta.core.pose.Pose()    # a temporary pose to export to PyMOL                                                                     
temp_pose.assign(p)
counter = 0    # for pretty output to PyMOL                                                                                     

while not jd.job_complete:
    
    counter += 1
    
    test_pose = rosetta.core.pose.Pose()
    # set staring pose to input pose
    test_pose.assign(p)    
    # change pose name for pretty viewing in PyMol
    test_pose.pdb_info().name(job_output + '_' + str(counter))

    # apply mc mover (with all of the stuff)
    mc.apply(test_pose)
    
    # have the jd output the resulting low energy model
    jd.output_decoy(test_pose)

    # see if this works the way I want it to in the loop
    #test_pose.dump_pdb('out.pdb')
    #viewmyprotein('out.pdb',"%s" %("2"+str(counter)))
    #viewmyprotein('out.pdb')



Built with

RISE https://github.com/damianavila/RISE

Jupyter

PyRosetta4

<img src="10_21_2016_group_meeting.png" />

In [ ]:
from IPython.display import HTML

In [ ]:
javascript = """
<script type='text/javascript' src='bower_components/bio-pv/bio-pv.min.js'>

<script>
var parent = document.getElementById('viewer');
var viewer = bio-pv.Viewer(parent,
                      { width : 300, height : 300, antialias : true });
bio-pv.io.fetchPdb('bagel-foldit/bagel.pdb', function(structure) {
  // select the two ligands contained in the methyl transferase by name, so
  // we can display them as balls and sticks.
  viewer.on('viewerReady', function() {
    var ligand = structure.select('ligand'});
    viewer.ballsAndSticks('ligand', ligand);
    // display the whole protein as cartoon
    viewer.cartoon('protein', structure);

    // set camera orientation to pre-determined rotation, zoom and
    // center values that are optimal for this very protein
    var rotation = [
      0.1728139370679855, 0.1443438231945038,  0.974320650100708,
      0.0990324765443802, 0.9816440939903259, -0.162993982434272,
      -0.9799638390541077, 0.1246569454669952,  0.155347332358360
    ];
    var center = [6.514, -45.571, 2.929];
    viewer.setCamera(rotation, center, 73);
  });
});
</script>"""

In [ ]:
HTML(javascript)

In [ ]:
import pvviewer as pv
struct = pv.mol.from_file('bagel-foldit/bagel.pdb')
viewer = pv.Viewer()

viewer.cartoon('mytest',struct)
viewer.auto_zoom()
viewer.show()